In [157]:
import pandas as pd
import numpy as np
import glob
import os

from datetime import time
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [177]:
all_files = glob.glob(os.path.join('', '*.csv'))
dtypes = {'user_type':str, 'bike_share_for_all_trip':str, 'rental_access_method':str}

data = pd.concat((pd.read_csv(f, dtype=dtypes) for f in all_files), ignore_index=True)

In [179]:
data.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip,rental_access_method
0,75284,2018-01-31 22:52:35.2390,2018-02-01 19:47:19.8240,120.0,Mission Dolores Park,37.761420,-122.426435,285.0,Webster St at O'Farrell St,37.783521,-122.431158,2765,Subscriber,No,NaN
1,85422,2018-01-31 16:13:34.3510,2018-02-01 15:57:17.3100,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,2815,Customer,No,NaN
2,71576,2018-01-31 14:23:55.8890,2018-02-01 10:16:52.1160,304.0,Jackson St at 5th St,37.348759,-121.894798,296.0,5th St at Virginia St,37.325998,-121.877120,3039,Customer,No,NaN
3,61076,2018-01-31 14:53:23.5620,2018-02-01 07:51:20.5000,75.0,Market St at Franklin St,37.773793,-122.421239,47.0,4th St at Harrison St,37.780955,-122.399749,321,Customer,No,NaN
4,39966,2018-01-31 19:52:24.6670,2018-02-01 06:58:31.0530,74.0,Laguna St at Hayes St,37.776435,-122.426244,19.0,Post St at Kearny St,37.788975,-122.403452,617,Subscriber,No,NaN


In [180]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [181]:
data['start_time'] = pd.to_datetime(data['start_time'])

In [182]:
data['Business Day'] = (data['start_time'].dt.date.isin(pd.date_range(start='2018-01-01',end='2020-01-01', freq=us_bd).date))*1

In [183]:
data['Holiday'] = (data['start_time'].dt.date.isin(USFederalHolidayCalendar().holidays(start='2018-01-01',end='2020-01-01').date))*1

In [184]:
day_name = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
day_name = dict(enumerate(day_name))
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November']
month_name = dict(enumerate(month_name))
season_name = ['Spring', 'Summer', 'Fall']
season_name = dict(enumerate(season_name))

In [185]:
for i in range(3):
    data[season_name[i]] = (data['start_time'].dt.month%12 // 3 + 1 == i)*1

In [186]:
for i in range(6):
    data[day_name[i]] = (data['start_time'].dt.dayofweek == i)*1

In [187]:
for i in range(11):
    data[month_name[i]] = (data['start_time'].dt.month == i+1)*1

In [188]:
data['Hour'] = data['start_time'].dt.hour

In [189]:
data['Days Passed'] = (data['start_time'] - pd.to_datetime('2018-01-01')).dt.days

In [190]:
data['Commute In'] = ((data['start_time'].dt.time >= time(7,0)) & (data['start_time'].dt.time <= time(9,30)) & (data['Business Day']))*1

In [191]:
data['Commute Out'] = ((data['start_time'].dt.time >= time(16,30)) & (data['start_time'].dt.time <= time(18,30)) & (data['Business Day']))*1

In [ ]:
# autoregressive features are most easily added after we groupby and/or reindex by time and location, so we can use the shift() method to shift per hour
# data['Lag'] = data['Change in Bikes'].shift()